In [ ]:
## BEGIN
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', **{ "size":16}) #**{,,'serif':['Palatino']
rc('text', usetex=True)
import numpy.random as rnd
rnd.seed()
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import scipy.linalg as la
import pickle as cp
import scipy.optimize as opt
from scipy.optimize import curve_fit as fit
#import mps
import cProfile
import subprocess
from scipy.special import hyp2f1, erf
from scipy.optimize import brentq
from matplotlib import cm
cmap = cm.get_cmap("tab20")
import itertools

In [ ]:
def prepare_graph(L, cut_boundary = 0, **kwargs):
    xy2i = lambda x,y:x*L + y
    xs, ys =  np.arange(L**2) // L, np.arange(L**2) % L
    xp, xm = (xs + 1) % L, (xs - 1) % L
    yp, ym = (ys + 1) % L, (ys - 1) % L


    matrix = sp.coo_matrix((L**2, L**2), dtype= int)
    inds = np.array([ xy2i(xp, ys), xy2i(xm, ys), 
                     xy2i(xs , yp), xy2i(xs, ym)]).T.reshape(-1)
    data = np.ones_like(inds, dtype=int)
    indptr = np.arange(L ** 2 + 1) * 4
    full_graph = sp.csc_matrix((data, inds, indptr),shape=(L**2, L**2))

    ## cut all the boundaries
    if 0:
        lil = full_graph.tolil()
        lil[:L, -L:] = 0
        lil[-L:, :L] = 0
        lil[::L, L-1::L] = 0
        lil[L-1::L, ::L] = 0
        full_graph = lil.tocsc()
    ##############

    edges = sp.coo_matrix(full_graph)
    ut_ind = edges.row > edges.col
    edges = sp.coo_matrix((edges.data[ut_ind], (edges.row[ut_ind], edges.col[ut_ind])), shape = (L**2,L**2))
    #degree = np.diff(full_graph.indptr)
    return full_graph, edges

# Ising model on the square lattice

We will study numerically the Ising model on a 2D square lattice, using the Swendsen-Wang Monte Carlo algorithm. It is based on the Fortuin-Kasterleyn (FK) representation of the Ising model, which is interesting in its own right.

## Preparation: what is the critical temperature?

- Recall the high-$T$ and low-$T$ expansions of the Ising model on the 2D square lattice, whose partition function is 
$$ Z = \sum_{\{\sigma\} } \exp\left(\sum_{\left<ij\right>} K \sigma_i \sigma_j \right) \,.$$
Here $K=\beta J$ is the dimensionless temperature.

- Deduce the Kramers-Wagner duality relation, and identify the self-dual point $K_c$ as satisfying 
$$ e^{-2K_c} = \sqrt{2} - 1 \,. $$
This is the critical inverse temperature. 

##  Fortuin-Kasterleyn (FK) representation

This is a slightly different high-$T$ expansion of the Ising model. One starts by rewriting the Boltzmann weight on a bond as follows
$$ e^{K \sigma \sigma'} \propto (1 + x \delta_{\sigma, \sigma'}) \,,\, x = e^{2K}-1 $$
Doing this for all bonds, we obtain an expansion of the partition function as a sum of "FK cluster" configurations. 

- Show that the relative weight of a FK cluster configuration is 
$$ w^{\text{number of active bonds}} (1-w)^{\text{number of inactive bonds}} Q^{\text{number of clusters}} $$
where 
$$ w = \frac{x}{1+x} = 1-e^{-2K} $$
and $$Q = 2 \; \text{(Ising)}$$

The active bonds are the ones we chose the term $\delta_{\sigma, \sigma'}$; a (FK) cluster is a connected component of the graph formed by the active bonds. 

### Cluster algorithm (Swendsen Wang)
The FK representation defines a joint distribution of cluster and spin configurations, $(C, \sigma)$. We have the conditional probability

$$
P(\sigma | C) = Q^{-\text{number of clusters}} \,,\, 
$$

if the $\sigma$ is such that the sites of each FK cluster have the same spin. Otherwise $P(\sigma|C) = 0$. So it is easy to sample $\sigma$ given $C$: we choose a spin value for each FK cluster randomly and independently. 

Moreover it is also easy to sample $C$ given $\sigma$. By Bayes rule

$$
P(C | \sigma) \propto w^{\text{number of active bonds}} (1-w)^{\text{number of inactive bonds}} \,,\, 
$$

if the sites of each FK cluster have the same spin. Therefore, to sample $C$ given $\sigma$, 
- all the bonds with disaligned spin must be inactive;
- for each bond with aligned spin, it is acitve with probability $w$. 

The Monte Carlo algorithm starts from any configuration (say, of spin) $\sigma_0$, samples a cluster configuration $C_1$ conditioned on it, and then sample a new spin configuration $\sigma_1$ conditioned on $C_1$, so on and so forth. Compared to Metropolis Monte Carlo based on local updates, cluster algorithms (or loop algorithms) suffer less from the *critical slowdown*.


## Some nice pictures first

In [ ]:
lb = 6
L = 2 ** lb
full_graph, edges = prepare_graph(L)
spin = np.zeros(L**2, dtype=int)
w =  2 - 2 ** .5  # critical point 

In [ ]:
# do 50 Monte Carlo updates 
for j in range(50):

    same = (spin[edges.row] == spin[edges.col])  
    
    edges.data[:] = 0
    edges.data[same] = (rnd.rand(np.sum(same)) < w) # same color: throw a dice
   
    FK_graph = edges.tocsc()
    FK_graph.eliminate_zeros()
    n_cl, FK_clusters = sp.csgraph.connected_components(FK_graph, directed=0, return_labels=1)
    
    FK_spin = rnd.randint(2, size = n_cl) * 2 - 1
    spin[:] = FK_spin[FK_clusters]

In [ ]:
# FK clusters takes lots of time to plot

plot_FK = 1

plt.figure(1, figsize = (6,6))
plt.pcolor(spin.reshape(-1, L).T, cmap = "RdBu", vmin = -3, vmax = 1)
xs, ys =  np.arange(L**2) // L, np.arange(L**2) % L
if plot_FK:
    for k, (i,j) in enumerate(zip(edges.row, edges.col)):
        #diff = spin[1] 
        #d = edges.data[k]
        if FK_clusters[i] != FK_clusters[j]: continue

        z1, z2 = xs[i] +.5 + 1.j*(ys[i]+.5), (xs[j]+.5) + 1.j * (ys[j] + .5)
        if abs(z1 - z2) > 2: continue
        zm = (z1 +z2)/2
        w1 = (z1 - zm) * 1.j + zm
        w2 = 2 * zm - w1
        plt.plot([z1.real, z2.real], [z1.imag, z2.imag], c = cmap(FK_clusters[i]/n_cl))
plt.gca().set_aspect('equal')


## Spin spin correlation (two point function)

Measure the two spin-spin correlation function in different phases (and with different system sizes). 
Describe the results.

In [ ]:
res_2p = dict()

In [ ]:

def collect_data(w = [2 - 2 **.5], lbs = [6,7,8], N = 500, scale = "log", **kwargs):
    """
    Routine to collect MC data
    - w: coupling, from 0 (para) to 1 (ferro). Default: critical value
    - lbs: a list of lb(system sizes). Default: [6,7,8]
    - N: Monte Carlo update number. Default: 500
    - scale: "log" or "linear". Default: log
    """
    for lb in lbs:
        L = 2 ** lb
        print("L = %d" % L)
        full_graph, edges = prepare_graph(L)


        if not (w, lb) in res_2p:
            spin = rnd.randint(2, size = L ** 2)
            if scale == "log":
                rs = np.sort(np.hstack([2 ** np.arange(lb), 3 * 2 ** np.arange(lb-1)]))
            else:
                rs = np.arange(20)
            ans = np.zeros(len(rs)+1,)
            res_2p[w,lb] = (rs, ans, spin)
        else:
            rs, ans, spin = res_2p[w,lb]

        #get spin clusters
        for j in range(N):

            # start from 4-state
            same = (spin[edges.row] == spin[edges.col])  
            #red = (spin[edges.row] + spin[edges.col] == 3)
            edges.data[:] = 0
            edges.data[same] = (rnd.rand(np.sum(same)) < w) # same color: throw a dice
            #edges.data[red] = 1 # 
            FK_graph = edges.tocsc()
            FK_graph.eliminate_zeros()
            n_cl, FK_clusters = sp.csgraph.connected_components(FK_graph, directed=0, return_labels=1)

            FK_spin = rnd.randint(2, size = n_cl) * 2 - 1
            #FK_spin[np.unique(FK_clusters[ind_boundary])] *= 0
            spin[:] = FK_spin[FK_clusters]
            #print(n_cl, end = " ")

            # calculate 2p function
            if ans[0] == 0 and j < 50: continue
            for k,x in enumerate(rs):
                intersect = (spin.reshape(L,L) * np.roll(spin.reshape(L,L), x, axis = 1))
                ans[k+1] += (np.sum(intersect) / L ** 2)
            ans[0] += 1

### Paramagnetic phase

In [ ]:
w_p = 2 - 2 ** .5 - .1 # critical point
collect_data(w = w_p, lbs = [5,6,7], scale = "linear" )


In [ ]:
for (w, lb), (rs, ans, spin) in res_2p.items():
    if w != w_p: continue
    ys = ans[1:] / ans[0]
    errs = ans[1:] / ans[0] ** 1.5 * 2
    plt.errorbar(rs, ys , marker="o", label="$L=2^{%d}$" % lb)
    
    plt.xlabel("$r$")
    plt.ylabel(r"$\left< \sigma_{0,0} \sigma_{r,0} \right>$")
#plt.loglog(rs, rs ** (-1/4) / 1.5)
plt.legend()
#plt.yscale("log")
#plt.xscale("log")

### Ferromagnetic phase

In [ ]:
w_f = 2 - 2 ** .5 + .05 # critical point
collect_data(w = w_f, lbs = [5,6,7], scale = "linear" )


In [ ]:
for (w, lb), (rs, ans, spin) in res_2p.items():
    if w != w_f: continue
    ys = ans[1:] / ans[0]
    errs = ans[1:] / ans[0] ** 1.5 * 2
    plt.errorbar(rs, ys , marker="o", label="$L=2^{%d}$" % lb)
    
    plt.xlabel("$r$")
    plt.ylabel(r"$\left< \sigma_{0,0} \sigma_{r,0} \right>$")
#plt.loglog(rs, rs ** (-1/4) / 1.5)
plt.legend()
#plt.yscale("log")
#plt.xscale("log")

### Critical point

In [ ]:
w_c = 2 - 2 ** .5 # critical point
collect_data(w = w_c, lbs = [5,6,7], N = 2000  )

Comment on the two point functions in different system sizes below. It is known by exact solution that the scaling dimension of the spin operator is $\Delta_{\sigma} = 1/8$ (we will see that in a later TD). Let us discover how this is related to the behavior of the two point function.

In [ ]:

for (w, lb), (rs, ans, spin) in res_2p.items():
    if w != 2 - 2**.5: continue
    ys = ans[1:] / ans[0]
    errs = ans[1:] / ans[0] ** 1.5 * 2
    plt.errorbar(rs, ys  , yerr = errs, marker="o", label="$L=2^{%d}$" % lb)
    
    plt.xlabel("$r$")
    plt.ylabel(r"$\left< \sigma_{0,0} \sigma_{r,0} \right>$")
plt.loglog(rs, rs ** (-.125) / 1.5)
plt.legend()

The same data, plotted from a long-distance/field theory point of view. 
How do we collapse the data? 

In [ ]:
for (w, lb), (rs, ans, spin) in sorted(res_2p.items())[::-1]:
    if w != 2 - 2**.5: continue
    L =  2 ** lb
    ys = ans[1:] / ans[0]
    errs = ans[1:] / ans[0] ** 1.5 * 2
    plt.errorbar(rs / L, ys  * L ** (.25), yerr = errs * L ** (.25), marker="o", label="$L=2^{%d}$" % lb)
    
    #plt.loglog(rs / L, ans[1:] / ans[0] , "o-", label="$L=2^{%d}$" % lb )
   
    plt.xlabel("$r / L$")
    plt.ylabel(r"$\left<  \sigma_{0,0} \sigma_{r,0} \right>  L^{2\Delta}$")
plt.legend()
plt.xscale("log")
plt.yscale("log")

The quantity of the $y$ axis above is the two-point function of a *scaling operator*, which one may define "experimentally" as:
### $$ \sigma_{\text{scaling}}(x) = \sigma_{\text{lattice}}(r = x  L) L^{\Delta_{\sigma}}$$

One of the goals of this course is to understand how this formula emerges from the renormalization procedure.

The rescaling of the operator is necessary to guarantee that the correlation function between scaling operators are independent of the lattice spacing (*scale invariant*) at the critical point.

- Is this achievable away from the critical point?



## Epilogue: from Ising to percolation

- Show that the spin-spin correlation function is equal to the probability that the two sites are on the same FK cluster:

$$ \left< \sigma_x \sigma_{x'} \right> = \mathbb{P}(\text{$x$, $x'$ on the same FK cluster})$$

- Consider the FK partition function with $Q=1$. Argue that it describes bond percolation on the square lattice. What is the analogue of spin-spin correlation in percolation?